In [ ]:
#@title Student Information
Name = 'Lingxuan Ye' #@param {type:"string"}
Login_ID = '' #@param {type:"string"}
SIS_ID = 'value' #@param {type:"string"}

In [ ]:
import json

import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn.decomposition import PCA
import yfinance as yf

# may be not backward compatible (latest verson of Jupyter/Ipython)
from IPython.display import HTML, display  # comment this if `ImportError`
# from IPython.core.display import HTML, display  # uncomment if ...

In [ ]:
def debug(x):
    display(HTML(x.to_html()))

# Problem 1. US Covid Death Counts by States by Time (2 points)

The US CDC maintains a [dataset](https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36/data) tracking the US covid death counts by states by time. We would like to use this dataset to extract the (accumulative) total death counts by month for each state and put it into a pandas DataFrame. 

**Requirements**

1. Use this [link](https://data.cdc.gov/api/views/9mfq-cb36/rows.csv?accessType=DOWNLOAD) to download the dataset into your colab server machine and load it into a pandas DataFrame (i.e., the "original" DataFrame thereafter in this HW). **The downloading of data has to be performed programatically in your notebook, in another word, manually downloading it and then uploading it into your colab notebook is not acceptable**
2. Use the `tot_death` column for the data extraction
3. The result pandas DataFrame should contain the following columns:
 - `state`: the `state` column of the original DataFrame
 - `year`: the year of the `submission_date` column in the original DataFrame
 - `month`: the month of the `submission_date` column in the original DataFrame
 - `tot_death`: the `tot_death` in the original DataFrame
4. For each state for each month, there should be only one row in the result pandas DataFrame that contains the total death counts for that state in that month.  

In [ ]:
!curl "https://data.cdc.gov/api/views/9mfq-cb36/rows.csv?accessType=DOWNLOAD" > _covid.csv
# !curl "https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json" > _geo.json
# !curl "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json" > _geo.json
# !curl "https://gist.githubusercontent.com/wavded/1250983/raw/bf7c1c08f7b1596ca10822baeb8049d7350b0a4b/stateCodeToFips.json" > _fips.json

In [ ]:
raw = pd.read_csv('_covid.csv')
HTML(
    f"""
    <details>
        <summary>
        Expand for more info ...
        </summary>
        {raw._repr_html_()}
    </details>
    """
)

In [ ]:
data = raw[['submission_date', 'state', 'tot_death']]
month_year = data['submission_date'].str.extract(r'^(.+?)(?=\/).+?([^\/]+)$')
data.insert(0, 'year', month_year[1])
data.insert(1, 'month', month_year[0])

# `inplace=True` has some irritating warning,
# while statement below is memory-consuming
data_nested = data.drop(
    'submission_date', axis=1
).sort_values(['year', 'month']).groupby(['state', 'year', 'month']).last()

data_flatened = data_nested.reset_index()

display(
    HTML('<h2>Nested Data</h2>'),
    # HTML(data_nested.to_html()),  # uncomment it for details
    data_nested,
    HTML('<h2>Flatened Data</h2>'),
    # HTML(data_nested.to_html()),  # ditto
    data_flatened
)

# Problem 2. Animation of the Death Counts by States by Time (2 points)

Starting with the result pandas DataFrame derived from the Problem 1, generate a choropleth plot that shows the total death counts by state over time.  

**Requirements**

1. The plot has to be generated using the [`plotly.express.choropleth`](https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth.html) function.
2. The plot should features an animation UI by using the `animation_frame` argument of the plotting function. When the user plays the animation, the animation needs to start from the earliest time and display the death counts for each state month by month in the increasing order of time. 
3. The color range has to be fixed so that the plot won't dynamically change the color range during the animation.

## Notice

Some values of column 'state' are **INVALID**. For example:

- MP: I presume it stands for Military Police Corps, while it literally cannot have a fips code, let alone that it has been registered in geojson.
- NYC: Maybe New York City counts its case individually from New York State, while it is not state-level administrative division I think. So I add the stats of NYC to NY.
- FSM: If FSM does not refer to Flying Spaghetti Monster, then it is most likely to be Federated States of Micronesia, which even not a US territory!

Maybe there are more invalid value that I did not know, I will drop them anyway.

In [ ]:
data = data_flatened.replace('NYC', 'NY').groupby(
    ['state', 'year', 'month']
).sum().reset_index()

In [ ]:
px.choropleth(
    data,
    locations='state',
    locationmode="USA-states",
    color='tot_death',
    scope="usa",
    animation_frame=(data['year'] + '/' + data['month']),
    range_color=(data['tot_death'].min(), data['tot_death'].max())
)

# Problem 3. PCA of Major US Indices (2 points)

1. Generate a seaborn pairplot of the PCA scores of the daily close prices from 2017-01-01 to 2022-11-25 of the four major US financial market indicies: `SP500`, `DOW`, `NASDAQ`, and `RUSSELL2000` (1 point).

2. To retain 99% of the data variability, how many PC comoponents do you need to retain (1 point)?

**Requirements**

1. Use the Yahoo finance API to prepare the data

**Notes**

1. The symbols for the indices in the Yahoo finance API: `SP500` - `^GSPC`, `DOW` - `^DJI`, `NASDAQ` - `^IXIC`, `RUSSELL2000` - `^RUT`

## 1

In [ ]:
COLUMNS = ['^GSPC', '^DJI', '^IXIC', '^RUT']
data = yf.download(
  COLUMNS,
  start='2017-01-01',
  end='2022-11-26',
  progress=False
)['Close']

In [ ]:

pca = PCA()
scores = pd.DataFrame(
    pca.fit_transform(data),
    columns=(f'PC {i + 1}' for i in range(len(COLUMNS)))
)

sns.pairplot(scores)

## 2

In [ ]:
cumsum = pca.explained_variance_ratio_.cumsum()

In [ ]:
for i, j in enumerate(cumsum):
    if j >= 0.99:
        print(f'number of pca components to retain: {i + 1}')
        break